In [1]:
import pandas as pd
import requests
import json

In [2]:
url_data = "https://sdmx.oecd.org/public/rest/data/OECD.ENV.EPI,DSD_GG@DF_ECO_OP,1.0/TZA+ETH+UGA+KEN.A.TECHPAT_RD+EGS+IFF_ODA+ENVTAX_VEH+ENGTAX+ENVTAX+ENVTAXTRANS+FPRICE_DIE+FPRICE_PET+EPRICE_RES+EPRICE_IND+REGMNG+GPAT_DE+TECHPAT_PAT+ECOPOL.INV_10P6HB+USD_PPP_L+PT_INV_D+PT_INV_W_ENV+PT_B1GQ+PT_TAX_REV+PT_TAX_REV_ENV+USD_PPP_KWH.?startPeriod=2017&endPeriod=2022&dimensionAtObservation=AllDimensions"

In [3]:
headers = {'Accept': 'application/vnd.sdmx.data+json; charset=utf-8; version=1.0'}

In [4]:
# url_struct = "https://sdmx.oecd.org/public/rest/dataflow/OECD.ENV.EPI/DSD_GG@DF_ECO_OP/1.0?references=all"
# headers_struct = {'Accept': 'application/vnd.sdmx.structure+json; charset=utf-8; version=1.0'}

In [5]:
# file = requests.get(url_struct, headers=headers_struct)

In [6]:
file_data = requests.get(url_data, headers=headers)

In [8]:
file_data

<Response [200]>

In [172]:
def print_keyjson(file_json):
    if (file_json.status_code == 200):
        responseJson = file_json.json()
        dataJson = responseJson.get('data')
        metaJson = responseJson.get('meta')
        errorJson = responseJson.get('error')

        print("The query ID is: ", metaJson['id'])
        print("The dataset was prepared on: ", metaJson['prepared'])

In [173]:
print_keyjson(file_data)

The query ID is:  IREF000110
The dataset was prepared on:  2023-08-24T06:43:19Z


In [174]:
def create_df_temp(response):
    # Data transformation

    if (response.status_code == 200):

        responseJson = response.json()
        dataJson = responseJson.get('data')

        obsList = dataJson.get('dataSets')[0].get('observations')

        if (len(obsList) > 0):

            print('Data downloaded from %s' % response.url)

            timeList = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'TIME_PERIOD'][0]['values']
            refArea = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'REF_AREA'][0]['values']
            freqList = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'FREQ'][0]['values']
            subjectList = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'ACTIVITY'][0]['values']
            measureList = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'MEASURE'][0]['values']
            unitMeasureList = [item for item in dataJson.get('structure').get('dimensions').get('observation') if item['id'] == 'UNIT_MEASURE'][0]['values']

            obs = pd.DataFrame(obsList).transpose()
            obs.rename(columns = {0: 'series'}, inplace = True)
            obs['id'] = obs.index
            obs = obs[['id', 'series']]
            obs['dimensions'] = obs.apply(lambda x: re.findall('\d+', x['id']), axis = 1)
            obs['ref_area'] = obs.apply(lambda x: refArea[int(x['dimensions'][0])]['id'], axis = 1)
            obs['frequency'] = obs.apply(lambda x: freqList[int(x['dimensions'][1])]['id'], axis = 1)
            obs['subject'] = obs.apply(lambda x: subjectList[int(x['dimensions'][4])]['id'], axis = 1)
            obs['measure'] = obs.apply(lambda x: measureList[int(x['dimensions'][2])]['id'], axis = 1)
            obs['unit_measure'] = obs.apply(lambda x: unitMeasureList[int(x['dimensions'][3])]['id'], axis = 1)
            obs['time'] = obs.apply(lambda x: timeList[int(x['dimensions'][5])]['id'], axis = 1)
            obs['names'] = obs['subject'] + '_' + obs['measure']

            data = obs.pivot_table(index = ['ref_area', 'unit_measure'], columns = ['time'], values = 'series')
            
            return (obs, data)

In [175]:
temp_df, _ = create_df_temp(file_data)
temp_df

Data downloaded from https://sdmx.oecd.org/public/rest/data/OECD.ENV.EPI,DSD_GG@DF_ECO_OP,1.0/TZA+ETH+UGA+KEN.A.TECHPAT_RD+EGS+IFF_ODA+ENVTAX_VEH+ENGTAX+ENVTAX+ENVTAXTRANS+FPRICE_DIE+FPRICE_PET+EPRICE_RES+EPRICE_IND+REGMNG+GPAT_DE+TECHPAT_PAT+ECOPOL.INV_10P6HB+USD_PPP_L+PT_INV_D+PT_INV_W_ENV+PT_B1GQ+PT_TAX_REV+PT_TAX_REV_ENV+USD_PPP_KWH.?startPeriod=2017&endPeriod=2022&dimensionAtObservation=AllDimensions


,id,series,dimensions,ref_area,frequency,subject,measure,unit_measure,time,names
0:0:0:0:0:0,0:0:0:0:0:0,1.99,"[0, 0, 0, 0, 0, 0]",KEN,A,_T,FPRICE_DIE,USD_PPP_L,2017,_T_FPRICE_DIE
0:0:0:0:0:1,0:0:0:0:0:1,2.21,"[0, 0, 0, 0, 0, 1]",KEN,A,_T,FPRICE_DIE,USD_PPP_L,2018,_T_FPRICE_DIE
0:0:0:0:0:2,0:0:0:0:0:2,2.08,"[0, 0, 0, 0, 0, 2]",KEN,A,_T,FPRICE_DIE,USD_PPP_L,2019,_T_FPRICE_DIE
0:0:0:0:0:3,0:0:0:0:0:3,1.81,"[0, 0, 0, 0, 0, 3]",KEN,A,_T,FPRICE_DIE,USD_PPP_L,2020,_T_FPRICE_DIE
0:0:0:0:0:4,0:0:0:0:0:4,1.97,"[0, 0, 0, 0, 0, 4]",KEN,A,_T,FPRICE_DIE,USD_PPP_L,2021,_T_FPRICE_DIE
...,...,...,...,...,...,...,...,...,...,...
3:0:0:0:0:2,3:0:0:0:0:2,2.86,"[3, 0, 0, 0, 0, 2]",UGA,A,_T,FPRICE_DIE,USD_PPP_L,2019,_T_FPRICE_DIE
3:0:0:0:0:3,3:0:0:0:0:3,2.54,"[3, 0, 0, 0, 0, 3]",UGA,A,_T,FPRICE_DIE,USD_PPP_L,2020,_T_FPRICE_DIE
3:0:2:2:0:0,3:0:2:2:0:0,15.35,"[3, 0, 2, 2, 0, 0]",UGA,A,_T,ENVTAX,PT_TAX_REV,2017,_T_ENVTAX
3:0:2:2:0:1,3:0:2:2:0:1,15.56,"[3, 0, 2, 2, 0, 1]",UGA,A,_T,ENVTAX,PT_TAX_REV,2018,_T_ENVTAX
